### 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

### 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

### 3. Extract Keypoint Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

Run when loading model

In [7]:
DATA_PATH = 'NEWW_Data'
actions = actions = np.array(['car', 'friend', 'mom', 'name','what'])
no_sequences = 40
sequence_length = 40

### 4. Setup Folders for Collection

In [7]:
# Define the path to the 'MP_Data' directory
DATA_PATH = 'NEWW_Data'

# Actions that we try to detect
actions = np.array(['car', 'friend', 'mom', 'name','what'])


no_sequences = 40

sequence_length = 40

# Create the 'MP_Data' directory if it doesn't exist
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Create subdirectories for each action
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    os.makedirs(action_path, exist_ok=True)  # 'exist_ok=True' prevents an error if the directory already exists

    # Create subdirectories for sequences within each action
    for sequence in range(0, no_sequences):
        sequence_path = os.path.join(action_path, str(sequence))
        os.makedirs(sequence_path, exist_ok=True)

In [8]:
def process_video(video_file, action):
    cap = cv2.VideoCapture(video_file)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        # NEW LOOP
        # Loop through actions
        # for action in actions:
            # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                    # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
            
            # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully  
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

### 5. Collect Keypoint Values for Training and Testing

In [9]:
video_folder = 'signvideos'

# Iterate through all video files in the folder and process each one
i = 0
for video_file in os.listdir(video_folder):
    if video_file.endswith('.mp4'):
        video_path = os.path.join(video_folder, video_file)
        process_video(video_path,actions[i])
        i+=1

### 6. Preprocess Data and Create Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'car': 0, 'friend': 1, 'mom': 2, 'name': 3, 'what': 4}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
print(np.array(sequences).shape) 
print(np.array(labels).shape)

(200, 40, 1662)
(200,)


In [16]:
X = np.array(sequences)
print(X.shape)

(200, 40, 1662)


In [17]:
y = to_categorical(labels).astype(int)
# labels  = np.array(labels)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
y_test.shape

(60, 5)

### 7. Build and Train LSTM Neural Network

In [54]:
del model

In [9]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from keras.regularizers import l2

In [21]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# model = Sequential()

# model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(40, 1662), kernel_regularizer=l2(0.001)))
# model.add(Dropout(0.3)) 

# model.add(BatchNormalization())

# model.add(LSTM(64, return_sequences=True, activation='relu', kernel_regularizer=l2(0.001)))
# model.add(Dropout(0.3))  

# model.add(BatchNormalization())

# model.add(LSTM(32, return_sequences=False, activation='relu', kernel_regularizer=l2(0.001)))
# model.add(Dropout(0.3))  

# model.add(BatchNormalization())

# model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
# model.add(Dropout(0.3))  

# model.add(BatchNormalization())

# model.add(Dense(actions.shape[0], activation='softmax'))

# early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# out = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping, tb_callback])


goodmodel1

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(40, 1662), kernel_regularizer=l2(0.001)))


model.add(BatchNormalization())


model.add(LSTM(64, return_sequences=True, activation='relu', kernel_regularizer=l2(0.001)))


model.add(BatchNormalization())


model.add(LSTM(32, return_sequences=False, activation='relu', kernel_regularizer=l2(0.001)))


model.add(BatchNormalization())


model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))


model.add(BatchNormalization())


model.add(Dense(actions.shape[0], activation='softmax'))


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)


model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


out = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping, tb_callback])


Epoch 1/100
5/5 [==============================] - 8s 226ms/step - loss: 2.5864 - accuracy: 0.1643 - val_loss: 2.7122 - val_accuracy: 0.1833
Epoch 2/100
5/5 [==============================] - 0s 75ms/step - loss: 1.7579 - accuracy: 0.4786 - val_loss: 2.2621 - val_accuracy: 0.2000
Epoch 3/100
5/5 [==============================] - 0s 70ms/step - loss: 1.4545 - accuracy: 0.6429 - val_loss: 2.1949 - val_accuracy: 0.2333
Epoch 4/100
5/5 [==============================] - 0s 72ms/step - loss: 0.9816 - accuracy: 0.8500 - val_loss: 2.2117 - val_accuracy: 0.2333
Epoch 5/100
5/5 [==============================] - 0s 75ms/step - loss: 0.8391 - accuracy: 0.8929 - val_loss: 2.0903 - val_accuracy: 0.2167
Epoch 6/100
5/5 [==============================] - 0s 74ms/step - loss: 0.7038 - accuracy: 0.9429 - val_loss: 2.2971 - val_accuracy: 0.1833
Epoch 7/100
5/5 [==============================] - 0s 70ms/step - loss: 0.6085 - accuracy: 0.9786 - val_loss: 2.0581 - val_accuracy: 0.2167
Epoch 8/100
5/5 [==

In [ ]:
plt.plot(out.history['loss'])
plt.plot(out.history['val_loss'])

In [16]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 40, 64)            442112    
                                                                 
 batch_normalization_5 (Bat  (None, 40, 64)            256       
 chNormalization)                                                
                                                                 
 lstm_22 (LSTM)              (None, 40, 128)           98816     
                                                                 
 batch_normalization_6 (Bat  (None, 40, 128)           512       
 chNormalization)                                                
                                                                 
 lstm_23 (LSTM)              (None, 64)                49408     
                                                                 
 batch_normalization_7 (Bat  (None, 64)               

### 8. Make Predictions

In [45]:
res = model.predict(X_test)
print(actions[np.argmax(res[0])])
print(actions[np.argmax(y_test[0])])

2/2 [==============================] - 1s 20ms/step
mom
mom


### 9. Save Weights

In [257]:
model.save('goodmodel3.h5')

c:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# model.load_weights('newmodel.h5')
model = tensorflow.keras.models.load_model('goodmodel1.h5')

### 10. Evaluation using Confusion Matrix and Accuracy

In [47]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [48]:
yhat = model.predict(X_test)

2/2 [==============================] - 0s 19ms/step


In [49]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [50]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[50,  0],
        [ 9,  1]],

       [[46,  0],
        [ 0, 14]],

       [[23, 26],
        [ 0, 11]],

       [[46,  0],
        [14,  0]],

       [[48,  1],
        [ 4,  7]]], dtype=int64)

In [51]:
accuracy_score(ytrue, yhat)

0.55

### 11. Test in Real Time

In [11]:
colors = [(245,117,16), (117,245,16), (16,117,245),(245,239,66), (245,66,200)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [12]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-40:]
        
        if len(sequence) == 40:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti